In [35]:
import numpy as np
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
base_path= "drive/My Drive/AI_LAWYER/brat-project-final/essay" # add essay number and ".txt" or ".ann"
total_essays=402

In [49]:
def getPathEnd(val):
    if val>=100:
        return str(val)
    elif val>=10:
        return "0"+str(val)
    return "00"+str(val)

def getTrainTest(val):
    return val.split(";")[1][1:-1] == "TRAIN"

In [75]:
df = pd.read_csv("drive/My Drive/AI_LAWYER/train-test-split.csv")
# df

In [76]:
test_data = []
train_data = []

In [77]:
k=0
for i in range(total_essays):
    total_path = base_path + getPathEnd(i+1)
    ann_file = open(total_path + ".ann", 'r')
    sentence = ann_file.read().split("\n")[-1]
    ann_file.close()
    if sentence!='':
        k+=1
print(k)
# all '.ann' files end with a blank line 

0


In [78]:
for i in range(total_essays):
    total_path = base_path + getPathEnd(i+1)
    ann_file = open(total_path + ".ann", 'r')
    sentences = ann_file.read().split("\n")[:-1]
    ann_file.close()
    _train_ = getTrainTest(df['ID;"SET"'][i])
    for sentence in sentences:
        temp_splits = sentence.split("\t")
        temp = {
            "essay_number":i+1,
            "node":temp_splits[0],
            "type":np.nan,
            "nodeType":np.nan,
            "startIndex":np.nan,
            "endIndex":np.nan,
            "sentence":np.nan,
            "Arg1":np.nan,
            "Arg2":np.nan,
            "message":np.nan
        }
    
        if temp_splits[0][0]=='T':
            temp_splits2 = temp_splits[1].split()
            temp["type"] = "node"
            temp["nodeType"] = temp_splits2[0]
            temp["startIndex"]  = int(temp_splits2[1])
            temp["endIndex"] = int(temp_splits2[2])
            temp["sentence"] = temp_splits[2]
        elif temp_splits[0][0]=='R':
            temp_splits2 = temp_splits[1].split()
            temp["Arg1"] = temp_splits2[1].split(':')[1]
            temp["Arg2"] = temp_splits2[2].split(':')[1]
            temp["type"] = "edge"
            temp["message"] = temp_splits2[0]
        else:
            temp["message"] = temp_splits[1]
            temp["type"] = "Stance"
        if _train_:
            train_data.append(temp)
        else:
            test_data.append(temp)
    

In [80]:
test_df = pd.DataFrame(test_data) 
test_df 

,essay_number,node,type,nodeType,startIndex,endIndex,sentence,Arg1,Arg2,message
0,4,T1,node,MajorClaim,262.0,376.0,this industry has affected the cultural attrib...,NaN,NaN,NaN
1,4,T2,node,MajorClaim,1663.0,1758.0,the tourism has created threatening pressure o...,NaN,NaN,NaN
2,4,T3,node,Claim,179.0,239.0,the tourism bring large profit for the destina...,NaN,NaN,NaN
3,4,A1,Stance,NaN,NaN,NaN,NaN,NaN,NaN,Stance T3 Against
4,4,T4,node,Claim,953.0,1031.0,international tourism can create negative impa...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2374,398,R7,edge,NaN,NaN,NaN,NaN,T14,T12,supports
2375,398,R8,edge,NaN,NaN,NaN,NaN,T11,T10,supports
2376,398,R9,edge,NaN,NaN,NaN,NaN,T9,T10,supports
2377,398,R10,edge,NaN,NaN,NaN,NaN,T15,T10,supports


In [81]:
train_df = pd.DataFrame(train_data) 
train_df 

,essay_number,node,type,nodeType,startIndex,endIndex,sentence,Arg1,Arg2,message
0,1,T1,node,MajorClaim,503.0,575.0,we should attach more importance to cooperatio...,NaN,NaN,NaN
1,1,T2,node,MajorClaim,2154.0,2231.0,a more cooperative attitudes towards life is m...,NaN,NaN,NaN
2,1,T3,node,Claim,591.0,714.0,"through cooperation, children can learn about ...",NaN,NaN,NaN
3,1,A1,Stance,NaN,NaN,NaN,NaN,NaN,NaN,Stance T3 For
4,1,T4,node,Premise,716.0,851.0,What we acquired from team work is not only ho...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9043,402,R10,edge,NaN,NaN,NaN,NaN,T13,T4,supports
9044,402,R11,edge,NaN,NaN,NaN,NaN,T9,T4,supports
9045,402,R12,edge,NaN,NaN,NaN,NaN,T10,T4,supports
9046,402,R13,edge,NaN,NaN,NaN,NaN,T14,T4,supports


In [85]:
test_data_file = open("drive/My Drive/AI_LAWYER/test_data.csv",'w')
test_df.to_csv(test_data_file)
test_data_file.close()

In [86]:
train_data_file = open("drive/My Drive/AI_LAWYER/train_data.csv",'w')
train_df.to_csv(train_data_file)
train_data_file.close()